In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import os
path = os.getcwd()
data_path = 'C:\\Users\\sunil\\Projects\\Dockship\\Exam Mark Prediction\\Dataset'
sub_path = 'C:\\Users\\sunil\\Projects\\Dockship\\Exam Mark Prediction\\Submissions'

#### Preprocessing ####
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, LeaveOneOut

le = LabelEncoder()
scaler = MinMaxScaler()

#### Models ####
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR

#### Evaluation ####
from sklearn.metrics import mean_squared_error

C:\Users\sunil\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [21]:
train = pd.read_csv(data_path+ '\\train.csv')
test = pd.read_csv(data_path+ '\\test.csv')
sample_sub = pd.read_csv(data_path+ '\\sample_submissions.csv')

In [22]:
train.drop('Unnamed: 0', axis = 1, inplace=True)
test.drop('Unnamed: 0', axis = 1, inplace=True)

---
# Preprocessing

In [23]:
df = pd.concat([train, test], axis = 0).reset_index(drop=True)

In [24]:
target = 'math score'

cat_cols = ['gender', 'ethnicity', 'parental level of education', 'lunch',
       'test preparation course', ]

num_cols = ['reading score', 'writing score']

In [25]:
#df[cat_cols] = df[cat_cols].apply(le.fit_transform)

df = pd.get_dummies(data=df, columns=cat_cols, drop_first=True)

---
# Train Test Split

In [26]:
train_proc, test_proc = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)

In [27]:
features = [col for col in test_proc.columns if col not in [target]]

In [28]:
trn, val = train_test_split(train_proc, test_size = 0.2, random_state = 1999)

##### Input for model
X_trn, X_val = trn[features], val[features]

##### Target column
y_trn, y_val = trn[target], val[target]

##### Features for test data that we will be predicting
X_test = test_proc[features]

#model = LGBMRegressor(n_estimators=700, learning_rate=0.007, max_depth=3, num_leaves = 100)
model = LGBMRegressor(n_estimators=700, learning_rate=0.007, max_depth=3, num_leaves = 100, random_state = 1)
model.fit(X_trn, y_trn)

preds = model.predict(X_val)

np.sqrt(mean_squared_error(y_val, preds))

xgb = XGBRegressor(random_state = 1, max_depth = 3, n_estimators=500, learning_rate = 0.03)
#model.fit(X_trn, y_trn)

#preds = model.predict(X_val)

#np.sqrt(mean_squared_error(y_val, preds))

model = CatBoostRegressor(random_state = 1)
model.fit(X_trn, y_trn, verbose=False)

preds = model.predict(X_val)

np.sqrt(mean_squared_error(y_val, preds))

In [29]:
cat = CatBoostRegressor(random_state = 1, max_depth = 3, n_estimators=500, learning_rate = 0.03)
cat.fit(X_trn, y_trn, verbose=False)

preds = cat.predict(X_val)

np.sqrt(mean_squared_error(y_val, preds))

5.143606378198863

rig = Ridge()

model = StackingRegressor(estimators = [('xgb', xgb),('cat', cat), ('Ridge', rig)])

model.fit(X_trn, y_trn)
preds = model.predict(X_test)
#np.sqrt(mean_squared_error(y_val, preds))

In [462]:
sample_sub[target] = np.round(preds)
sample_sub.to_csv(sub_path+'\\base_cross_val.csv', index=False)

ValueError: Length of values does not match length of index

# Cross Validation Strategy

In [30]:
def cross_val(model, train, test, features):
    
    N_SPLITS = 6
    
    oofs = np.zeros(len(train))
    preds = np.zeros(len(test))
    
    target_col = train[target]
    
    #folds = LeaveOneOut()
    folds = StratifiedKFold(n_splits = N_SPLITS, shuffle = True,random_state = 1)
    #stratified_target = pd.qcut( train[target], 10, labels=False, duplicates='drop')
    
    for i, (trn_idx, val_idx) in enumerate(folds.split(train[features], train[target])):
        print(f"===========================Fold {i+1}===========================")
        
        ######### Train Set #########
        X_trn, y_trn = train[features].iloc[trn_idx], target_col.iloc[trn_idx]
        
        ######### Validation Set ##########
        X_val, y_val = train[features].iloc[val_idx], target_col.iloc[val_idx]
        
        ######### Test Set #########
        X_test = test[features]
        
        ###### Model Fitting ######
        _ = model.fit(X_trn, y_trn)
        
        ###### Taking Predictions ######
        val_preds = model.predict(X_val)
        test_preds = model.predict(X_test)
        
        ###### Validating ######
        fold_score = np.sqrt(mean_squared_error(y_val, np.round(val_preds)))
        print(f'RMSE is {fold_score}')
        print()
        oofs[val_idx] = val_preds
        preds += test_preds/N_SPLITS
    
    total_error = np.sqrt(mean_squared_error(target_col, np.round(oofs)))
    print(f'RMSE on full dataset is {total_error}')
    
    return oofs, preds

In [31]:
%%time
oofs, preds = cross_val(CatBoostRegressor(random_state=1), train_proc, test_proc, features)

C:\Users\sunil\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)


===========================Fold 1===========================
Learning rate set to 0.035456
0:	learn: 14.9267536	total: 1.14ms	remaining: 1.14s
1:	learn: 14.5915754	total: 2.02ms	remaining: 1.01s
2:	learn: 14.2768742	total: 3.09ms	remaining: 1.03s
3:	learn: 13.9621590	total: 3.96ms	remaining: 987ms
4:	learn: 13.6772672	total: 4.86ms	remaining: 967ms
5:	learn: 13.3752953	total: 5.68ms	remaining: 941ms
6:	learn: 13.0904046	total: 6.52ms	remaining: 925ms
7:	learn: 12.8194206	total: 7.34ms	remaining: 910ms
8:	learn: 12.5616517	total: 8.17ms	remaining: 900ms
9:	learn: 12.3010925	total: 8.99ms	remaining: 890ms
10:	learn: 12.0780558	total: 9.8ms	remaining: 881ms
11:	learn: 11.8229404	total: 10.6ms	remaining: 875ms
12:	learn: 11.5850469	total: 11.4ms	remaining: 868ms
13:	learn: 11.3513850	total: 12.2ms	remaining: 860ms
14:	learn: 11.1434355	total: 13ms	remaining: 851ms
15:	learn: 10.9592219	total: 13.8ms	remaining: 850ms
16:	learn: 10.7674031	total: 14.6ms	remaining: 847ms
17:	learn: 10.5716707

179:	learn: 4.7992507	total: 139ms	remaining: 632ms
180:	learn: 4.7958013	total: 139ms	remaining: 631ms
181:	learn: 4.7913644	total: 140ms	remaining: 629ms
182:	learn: 4.7887788	total: 141ms	remaining: 628ms
183:	learn: 4.7838407	total: 141ms	remaining: 627ms
184:	learn: 4.7802664	total: 142ms	remaining: 626ms
185:	learn: 4.7760244	total: 143ms	remaining: 625ms
186:	learn: 4.7708464	total: 144ms	remaining: 624ms
187:	learn: 4.7652784	total: 144ms	remaining: 623ms
188:	learn: 4.7616602	total: 145ms	remaining: 622ms
189:	learn: 4.7581717	total: 146ms	remaining: 621ms
190:	learn: 4.7541174	total: 146ms	remaining: 620ms
191:	learn: 4.7453109	total: 147ms	remaining: 619ms
192:	learn: 4.7401466	total: 148ms	remaining: 618ms
193:	learn: 4.7360547	total: 148ms	remaining: 617ms
194:	learn: 4.7318356	total: 149ms	remaining: 615ms
195:	learn: 4.7264769	total: 150ms	remaining: 614ms
196:	learn: 4.7173584	total: 150ms	remaining: 613ms
197:	learn: 4.7141150	total: 151ms	remaining: 612ms
198:	learn: 

398:	learn: 3.9907732	total: 292ms	remaining: 439ms
399:	learn: 3.9884136	total: 292ms	remaining: 438ms
400:	learn: 3.9866970	total: 293ms	remaining: 438ms
401:	learn: 3.9826749	total: 294ms	remaining: 437ms
402:	learn: 3.9804429	total: 294ms	remaining: 436ms
403:	learn: 3.9763161	total: 295ms	remaining: 435ms
404:	learn: 3.9738200	total: 296ms	remaining: 435ms
405:	learn: 3.9716181	total: 297ms	remaining: 434ms
406:	learn: 3.9662704	total: 297ms	remaining: 433ms
407:	learn: 3.9617927	total: 298ms	remaining: 432ms
408:	learn: 3.9589234	total: 299ms	remaining: 432ms
409:	learn: 3.9547284	total: 299ms	remaining: 431ms
410:	learn: 3.9493700	total: 300ms	remaining: 430ms
411:	learn: 3.9473535	total: 301ms	remaining: 429ms
412:	learn: 3.9462072	total: 302ms	remaining: 429ms
413:	learn: 3.9416539	total: 302ms	remaining: 428ms
414:	learn: 3.9397920	total: 303ms	remaining: 427ms
415:	learn: 3.9383941	total: 304ms	remaining: 426ms
416:	learn: 3.9342278	total: 304ms	remaining: 425ms
417:	learn: 

621:	learn: 3.3851905	total: 448ms	remaining: 272ms
622:	learn: 3.3837090	total: 448ms	remaining: 271ms
623:	learn: 3.3829685	total: 449ms	remaining: 271ms
624:	learn: 3.3799870	total: 450ms	remaining: 270ms
625:	learn: 3.3790016	total: 450ms	remaining: 269ms
626:	learn: 3.3777434	total: 451ms	remaining: 268ms
627:	learn: 3.3746031	total: 452ms	remaining: 268ms
628:	learn: 3.3700033	total: 452ms	remaining: 267ms
629:	learn: 3.3684809	total: 453ms	remaining: 266ms
630:	learn: 3.3674803	total: 454ms	remaining: 265ms
631:	learn: 3.3647353	total: 454ms	remaining: 265ms
632:	learn: 3.3625421	total: 455ms	remaining: 264ms
633:	learn: 3.3619655	total: 456ms	remaining: 263ms
634:	learn: 3.3601872	total: 457ms	remaining: 262ms
635:	learn: 3.3592820	total: 457ms	remaining: 262ms
636:	learn: 3.3569646	total: 458ms	remaining: 261ms
637:	learn: 3.3539717	total: 459ms	remaining: 260ms
638:	learn: 3.3531152	total: 459ms	remaining: 260ms
639:	learn: 3.3500824	total: 460ms	remaining: 259ms
640:	learn: 

...

119:	learn: 5.1210168	total: 85.2ms	remaining: 625ms
120:	learn: 5.1130081	total: 85.9ms	remaining: 624ms
121:	learn: 5.1060482	total: 86.5ms	remaining: 623ms
122:	learn: 5.0962704	total: 87.3ms	remaining: 622ms
123:	learn: 5.0872275	total: 87.9ms	remaining: 621ms
124:	learn: 5.0817235	total: 88.5ms	remaining: 619ms
125:	learn: 5.0710722	total: 89.1ms	remaining: 618ms
126:	learn: 5.0645157	total: 89.8ms	remaining: 617ms
127:	learn: 5.0577034	total: 90.5ms	remaining: 617ms
128:	learn: 5.0505773	total: 91.2ms	remaining: 616ms
129:	learn: 5.0451568	total: 91.7ms	remaining: 614ms
130:	learn: 5.0389441	total: 92.4ms	remaining: 613ms
131:	learn: 5.0326865	total: 93ms	remaining: 612ms
132:	learn: 5.0275512	total: 93.7ms	remaining: 611ms
133:	learn: 5.0199737	total: 94.3ms	remaining: 610ms
134:	learn: 5.0094804	total: 95ms	remaining: 609ms
135:	learn: 5.0034526	total: 95.7ms	remaining: 608ms
136:	learn: 4.9948779	total: 96.3ms	remaining: 607ms
137:	learn: 4.9888062	total: 97ms	remaining: 606ms

348:	learn: 4.1033134	total: 240ms	remaining: 447ms
349:	learn: 4.1020655	total: 240ms	remaining: 446ms
350:	learn: 4.0992268	total: 241ms	remaining: 446ms
351:	learn: 4.0956294	total: 242ms	remaining: 445ms
352:	learn: 4.0903893	total: 242ms	remaining: 444ms
353:	learn: 4.0897513	total: 243ms	remaining: 443ms
354:	learn: 4.0874445	total: 244ms	remaining: 443ms
355:	learn: 4.0869273	total: 244ms	remaining: 442ms
356:	learn: 4.0821569	total: 245ms	remaining: 441ms
357:	learn: 4.0776309	total: 246ms	remaining: 441ms
358:	learn: 4.0749314	total: 246ms	remaining: 440ms
359:	learn: 4.0730878	total: 247ms	remaining: 439ms
360:	learn: 4.0726019	total: 248ms	remaining: 439ms
361:	learn: 4.0671337	total: 249ms	remaining: 438ms
362:	learn: 4.0621366	total: 249ms	remaining: 437ms
363:	learn: 4.0579460	total: 250ms	remaining: 437ms
364:	learn: 4.0537592	total: 251ms	remaining: 436ms
365:	learn: 4.0484981	total: 251ms	remaining: 435ms
366:	learn: 4.0473827	total: 252ms	remaining: 435ms
367:	learn: 

569:	learn: 3.4391696	total: 394ms	remaining: 297ms
570:	learn: 3.4368281	total: 395ms	remaining: 297ms
571:	learn: 3.4351376	total: 396ms	remaining: 296ms
572:	learn: 3.4342446	total: 396ms	remaining: 295ms
573:	learn: 3.4329862	total: 397ms	remaining: 295ms
574:	learn: 3.4271525	total: 398ms	remaining: 294ms
575:	learn: 3.4242135	total: 398ms	remaining: 293ms
576:	learn: 3.4227350	total: 399ms	remaining: 293ms
577:	learn: 3.4190636	total: 400ms	remaining: 292ms
578:	learn: 3.4162562	total: 400ms	remaining: 291ms
579:	learn: 3.4136785	total: 401ms	remaining: 290ms
580:	learn: 3.4080787	total: 402ms	remaining: 290ms
581:	learn: 3.4069452	total: 402ms	remaining: 289ms
582:	learn: 3.4031950	total: 403ms	remaining: 288ms
583:	learn: 3.4011887	total: 404ms	remaining: 288ms
584:	learn: 3.3981415	total: 404ms	remaining: 287ms
585:	learn: 3.3957715	total: 405ms	remaining: 286ms
586:	learn: 3.3946543	total: 406ms	remaining: 286ms
587:	learn: 3.3926479	total: 407ms	remaining: 285ms
588:	learn: 

795:	learn: 2.9639772	total: 549ms	remaining: 141ms
796:	learn: 2.9613743	total: 549ms	remaining: 140ms
797:	learn: 2.9603752	total: 550ms	remaining: 139ms
798:	learn: 2.9592084	total: 551ms	remaining: 139ms
799:	learn: 2.9572628	total: 552ms	remaining: 138ms
800:	learn: 2.9556660	total: 552ms	remaining: 137ms
801:	learn: 2.9543571	total: 553ms	remaining: 137ms
802:	learn: 2.9527117	total: 554ms	remaining: 136ms
803:	learn: 2.9518255	total: 554ms	remaining: 135ms
804:	learn: 2.9485429	total: 555ms	remaining: 134ms
805:	learn: 2.9471502	total: 556ms	remaining: 134ms
806:	learn: 2.9442898	total: 557ms	remaining: 133ms
807:	learn: 2.9422486	total: 557ms	remaining: 132ms
808:	learn: 2.9404320	total: 558ms	remaining: 132ms
809:	learn: 2.9401513	total: 559ms	remaining: 131ms
810:	learn: 2.9381718	total: 559ms	remaining: 130ms
811:	learn: 2.9359501	total: 560ms	remaining: 130ms
812:	learn: 2.9340342	total: 561ms	remaining: 129ms
813:	learn: 2.9325844	total: 561ms	remaining: 128ms
814:	learn: 

RMSE is 5.974081952267963

RMSE on full dataset is 5.707138387268051
Wall time: 5.48 s


In [262]:
sample_sub[target] = np.round(preds)
sample_sub.to_csv(sub_path+'\\base_cross_val.csv', index=False)

---
# Feature Engineering

In [32]:
def join(train, test):
    df_ = pd.concat([train, test], axis=0).reset_index(drop=True)
    return df_

def split(df_):
    train_, test_ = df_[:train.shape[0]:], df_[train.shape[0]:].reset_index(drop=False)
    features_ = [col for col in df_.columns if col not in [target]]
    
    return train_, test_, features_

In [33]:
train_proc, test_proc = df[:train.shape[0]], df[train.shape[0]:].reset_index(drop=True)

In [34]:
df = join(train, test)

In [35]:
#df[cat_cols] = df[cat_cols].apply(le.fit_transform)

df = pd.get_dummies(data=df, columns=cat_cols, drop_first=True)

---
### 1. Binning Features

In [38]:
def binning(x):
    dictionary = {1 : [i for i in range(0, 11)],
                  1 : [i for i in range(11, 21)],
                  1 : [i for i in range(21, 31)],
                  2 : [i for i in range(31, 41)],
                  2 : [i for i in range(41, 51)],
                  2 : [i for i in range(51, 61)],
                  3 : [i for i in range(61, 71)],
                  3 : [i for i in range(71, 81)],
                  4 : [i for i in range(81, 91)],
                  5 : [i for i in range(91, 101)]}
    for i,j in dictionary.items():
        if x in j:
            break
    return i

In [39]:
df['Reading_Bins'] = df['reading score'].apply(lambda x: binning(x))
df['Writing_Bins'] = df['writing score'].apply(lambda x: binning(x))

In [40]:
df['Median_score_per_reading_bins'] = df.groupby('Reading_Bins')['math score'].transform('median')

In [42]:
train_proc, test_proc, features = split(df)

In [44]:
features = ['reading score',
 'writing score',
 'gender_male',
 'ethnicity_group B',
 'ethnicity_group C',
 'ethnicity_group D',
 'ethnicity_group E',
 "parental level of education_bachelor's degree",
 'parental level of education_high school',
 "parental level of education_master's degree",
 'parental level of education_some college',
 'parental level of education_some high school',
 'lunch_standard',
 'test preparation course_none',
 'Median_score_per_reading_bins']

In [45]:
%%time
oofs, preds = cross_val(CatBoostRegressor(random_state=1), train_proc, test_proc, features)

C:\Users\sunil\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:668: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=6.
  % (min_groups, self.n_splits)), UserWarning)


===========================Fold 1===========================
Learning rate set to 0.035456
0:	learn: 14.8962596	total: 1.26ms	remaining: 1.26s
1:	learn: 14.5515901	total: 2.18ms	remaining: 1.09s
2:	learn: 14.2819533	total: 3.01ms	remaining: 1s
3:	learn: 13.9677800	total: 3.85ms	remaining: 959ms
4:	learn: 13.6671213	total: 4.76ms	remaining: 947ms
5:	learn: 13.3372688	total: 5.54ms	remaining: 918ms
6:	learn: 13.0411317	total: 6.33ms	remaining: 898ms
7:	learn: 12.7753759	total: 7.13ms	remaining: 885ms
8:	learn: 12.4847394	total: 8.11ms	remaining: 893ms
9:	learn: 12.2421351	total: 8.9ms	remaining: 882ms
10:	learn: 11.9814404	total: 9.74ms	remaining: 876ms
11:	learn: 11.7531693	total: 10.6ms	remaining: 869ms
12:	learn: 11.5307852	total: 11.4ms	remaining: 864ms
13:	learn: 11.2997076	total: 12.2ms	remaining: 861ms
14:	learn: 11.0655148	total: 13ms	remaining: 856ms
15:	learn: 10.8699762	total: 14ms	remaining: 861ms
16:	learn: 10.6753410	total: 14.7ms	remaining: 849ms
17:	learn: 10.4706693	tota

180:	learn: 4.8175133	total: 137ms	remaining: 622ms
181:	learn: 4.8055848	total: 138ms	remaining: 621ms
182:	learn: 4.8002791	total: 139ms	remaining: 620ms
183:	learn: 4.7932646	total: 140ms	remaining: 619ms
184:	learn: 4.7915457	total: 140ms	remaining: 618ms
185:	learn: 4.7893054	total: 141ms	remaining: 617ms
186:	learn: 4.7832327	total: 142ms	remaining: 616ms
187:	learn: 4.7789833	total: 142ms	remaining: 615ms
188:	learn: 4.7708380	total: 143ms	remaining: 614ms
189:	learn: 4.7659071	total: 144ms	remaining: 613ms
190:	learn: 4.7617314	total: 144ms	remaining: 612ms
191:	learn: 4.7563718	total: 145ms	remaining: 611ms
192:	learn: 4.7529267	total: 146ms	remaining: 610ms
193:	learn: 4.7495141	total: 147ms	remaining: 609ms
194:	learn: 4.7454217	total: 147ms	remaining: 608ms
195:	learn: 4.7413313	total: 148ms	remaining: 607ms
196:	learn: 4.7374734	total: 149ms	remaining: 606ms
197:	learn: 4.7350343	total: 149ms	remaining: 605ms
198:	learn: 4.7311608	total: 150ms	remaining: 604ms
199:	learn: 

397:	learn: 3.9683959	total: 292ms	remaining: 441ms
398:	learn: 3.9663301	total: 292ms	remaining: 440ms
399:	learn: 3.9622473	total: 293ms	remaining: 440ms
400:	learn: 3.9587654	total: 294ms	remaining: 439ms
401:	learn: 3.9563072	total: 295ms	remaining: 438ms
402:	learn: 3.9554162	total: 295ms	remaining: 437ms
403:	learn: 3.9509387	total: 296ms	remaining: 437ms
404:	learn: 3.9495841	total: 297ms	remaining: 436ms
405:	learn: 3.9447432	total: 298ms	remaining: 435ms
406:	learn: 3.9430611	total: 298ms	remaining: 434ms
407:	learn: 3.9391068	total: 299ms	remaining: 434ms
408:	learn: 3.9388612	total: 300ms	remaining: 433ms
409:	learn: 3.9355315	total: 300ms	remaining: 432ms
410:	learn: 3.9332467	total: 301ms	remaining: 431ms
411:	learn: 3.9294036	total: 302ms	remaining: 431ms
412:	learn: 3.9271228	total: 302ms	remaining: 430ms
413:	learn: 3.9238531	total: 303ms	remaining: 429ms
414:	learn: 3.9234658	total: 304ms	remaining: 428ms
415:	learn: 3.9198207	total: 305ms	remaining: 428ms
416:	learn: 

608:	learn: 3.3951776	total: 446ms	remaining: 286ms
609:	learn: 3.3924615	total: 447ms	remaining: 286ms
610:	learn: 3.3888111	total: 447ms	remaining: 285ms
611:	learn: 3.3862678	total: 448ms	remaining: 284ms
612:	learn: 3.3849906	total: 449ms	remaining: 283ms
613:	learn: 3.3845251	total: 450ms	remaining: 283ms
614:	learn: 3.3813492	total: 450ms	remaining: 282ms
615:	learn: 3.3788111	total: 451ms	remaining: 281ms
616:	learn: 3.3766204	total: 452ms	remaining: 280ms
617:	learn: 3.3761574	total: 453ms	remaining: 280ms
618:	learn: 3.3739603	total: 453ms	remaining: 279ms
619:	learn: 3.3715270	total: 454ms	remaining: 278ms
620:	learn: 3.3679189	total: 455ms	remaining: 277ms
621:	learn: 3.3663363	total: 455ms	remaining: 277ms
622:	learn: 3.3641496	total: 456ms	remaining: 276ms
623:	learn: 3.3617894	total: 457ms	remaining: 275ms
624:	learn: 3.3593683	total: 457ms	remaining: 274ms
625:	learn: 3.3575805	total: 458ms	remaining: 274ms
626:	learn: 3.3552616	total: 459ms	remaining: 273ms
627:	learn: 

...

233:	learn: 4.5615468	total: 169ms	remaining: 553ms
234:	learn: 4.5554306	total: 170ms	remaining: 553ms
235:	learn: 4.5528741	total: 170ms	remaining: 552ms
236:	learn: 4.5516584	total: 171ms	remaining: 551ms
237:	learn: 4.5479582	total: 172ms	remaining: 550ms
238:	learn: 4.5412121	total: 172ms	remaining: 549ms
239:	learn: 4.5355308	total: 173ms	remaining: 548ms
240:	learn: 4.5323003	total: 174ms	remaining: 548ms
241:	learn: 4.5292823	total: 175ms	remaining: 547ms
242:	learn: 4.5233188	total: 175ms	remaining: 546ms
243:	learn: 4.5183604	total: 176ms	remaining: 545ms
244:	learn: 4.5177039	total: 177ms	remaining: 544ms
245:	learn: 4.5131007	total: 177ms	remaining: 543ms
246:	learn: 4.5071565	total: 178ms	remaining: 543ms
247:	learn: 4.5052224	total: 179ms	remaining: 542ms
248:	learn: 4.5025267	total: 180ms	remaining: 541ms
249:	learn: 4.5001077	total: 180ms	remaining: 541ms
250:	learn: 4.4940678	total: 181ms	remaining: 540ms
251:	learn: 4.4921060	total: 182ms	remaining: 539ms
252:	learn: 

447:	learn: 3.7642838	total: 323ms	remaining: 398ms
448:	learn: 3.7624570	total: 324ms	remaining: 398ms
449:	learn: 3.7589075	total: 325ms	remaining: 397ms
450:	learn: 3.7552363	total: 326ms	remaining: 396ms
451:	learn: 3.7515488	total: 326ms	remaining: 396ms
452:	learn: 3.7459617	total: 327ms	remaining: 395ms
453:	learn: 3.7440820	total: 328ms	remaining: 394ms
454:	learn: 3.7402252	total: 329ms	remaining: 394ms
455:	learn: 3.7390511	total: 330ms	remaining: 393ms
456:	learn: 3.7357396	total: 330ms	remaining: 392ms
457:	learn: 3.7326919	total: 331ms	remaining: 392ms
458:	learn: 3.7286746	total: 332ms	remaining: 391ms
459:	learn: 3.7252584	total: 332ms	remaining: 390ms
460:	learn: 3.7227316	total: 333ms	remaining: 389ms
461:	learn: 3.7208230	total: 334ms	remaining: 389ms
462:	learn: 3.7190122	total: 334ms	remaining: 388ms
463:	learn: 3.7164806	total: 335ms	remaining: 387ms
464:	learn: 3.7145435	total: 336ms	remaining: 386ms
465:	learn: 3.7122389	total: 337ms	remaining: 386ms
466:	learn: 

643:	learn: 3.2650946	total: 474ms	remaining: 262ms
644:	learn: 3.2608862	total: 474ms	remaining: 261ms
645:	learn: 3.2590317	total: 475ms	remaining: 260ms
646:	learn: 3.2544799	total: 476ms	remaining: 260ms
647:	learn: 3.2509180	total: 477ms	remaining: 259ms
648:	learn: 3.2501644	total: 478ms	remaining: 258ms
649:	learn: 3.2476799	total: 478ms	remaining: 258ms
650:	learn: 3.2459922	total: 479ms	remaining: 257ms
651:	learn: 3.2431212	total: 480ms	remaining: 256ms
652:	learn: 3.2420265	total: 481ms	remaining: 255ms
653:	learn: 3.2406029	total: 481ms	remaining: 255ms
654:	learn: 3.2372400	total: 482ms	remaining: 254ms
655:	learn: 3.2334348	total: 483ms	remaining: 253ms
656:	learn: 3.2329562	total: 483ms	remaining: 252ms
657:	learn: 3.2322578	total: 484ms	remaining: 252ms
658:	learn: 3.2290673	total: 485ms	remaining: 251ms
659:	learn: 3.2259722	total: 486ms	remaining: 250ms
660:	learn: 3.2239692	total: 486ms	remaining: 249ms
661:	learn: 3.2215370	total: 487ms	remaining: 249ms
662:	learn: 

840:	learn: 2.8848665	total: 624ms	remaining: 118ms
841:	learn: 2.8824667	total: 624ms	remaining: 117ms
842:	learn: 2.8814064	total: 625ms	remaining: 116ms
843:	learn: 2.8796655	total: 626ms	remaining: 116ms
844:	learn: 2.8767304	total: 627ms	remaining: 115ms
845:	learn: 2.8754461	total: 628ms	remaining: 114ms
846:	learn: 2.8739377	total: 628ms	remaining: 113ms
847:	learn: 2.8728808	total: 629ms	remaining: 113ms
848:	learn: 2.8721022	total: 630ms	remaining: 112ms
849:	learn: 2.8707403	total: 630ms	remaining: 111ms
850:	learn: 2.8704486	total: 631ms	remaining: 110ms
851:	learn: 2.8694825	total: 632ms	remaining: 110ms
852:	learn: 2.8690360	total: 632ms	remaining: 109ms
853:	learn: 2.8663855	total: 633ms	remaining: 108ms
854:	learn: 2.8643805	total: 634ms	remaining: 107ms
855:	learn: 2.8626940	total: 635ms	remaining: 107ms
856:	learn: 2.8609563	total: 636ms	remaining: 106ms
857:	learn: 2.8594229	total: 636ms	remaining: 105ms
858:	learn: 2.8590878	total: 637ms	remaining: 105ms
859:	learn: 

RMSE is 5.982013269679422

RMSE on full dataset is 5.737844294058278
Wall time: 5.62 s


In [ ]:
sam